In [ ]:
# Notes

# Environment.yml for deploy
# Create tiffs from cubes and clip to overlap using the warp/translate
# Plot axes/label axes
# Function documentation
# Bin script tiff creation
# Analysis bin script
    # - Quiver png
    # - Homography.txt (9 numbers)
    # - Dataframe to csv (keep names the same)
    # - Stats to csv (Dataframe describe data + quantile data(x, y, magnitude))

# Box to Histogram
# Correlation/Coregistration in Z dimension
    # - Apply homography
    # - Difference the two DTMs for some Z offset

#-------Nice To Have-------
# Specify meters or pixels for plotting

In [10]:
import os
import sys
import cv2
import json
import gdal
import affine
from osgeo import ogr

import numpy as np
import pandas as pd

from plio.geofuncs import geofuncs
from plio.io.io_gdal import GeoDataset
from autocnet.matcher import subpixel as sp
from autocnet.transformation import homography as hg

import matplotlib.pyplot as plt
%matplotlib inline

plt.rcParams['image.cmap'] = 'plasma'

In [11]:
# setup the paths to cubes and tiffs
hirise_cub1 = "ESP_023524_1985_1m_o_isis3.cub"
hirise_cub2 = "ESP_048908_1985_1m_o_isis3.cub"
ctx_cub1 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_20m_o.cub"
ctx_cub2 = "J03_045994_1986_XN_18N282W_v6_20m_o.cub"
ctx_cub3 = "F05_037607_2008_XN_20N282W_v6_PosAndVelAndAngles_6m_o.cub"
ctx_cub4 = "J03_045994_1986_XN_18N282W_v6_6m_o.cub"
hrsc_cub = "H5270_0000_ND4.IMG"

In [12]:
# basepath = '/Volumes/Blueman/HiRISE_Jezero/'
# basepath = '/Volumes/Blueman/CTX_Jezero/20m_ctx/'
basepath = '/Volumes/Blueman/CTX_Jezero/6m_ctx/'
# basepath = '/Volumes/Blueman/HRSC_Jezero/'

cub_image1 = os.path.join(basepath, ctx_cub3)
cub_image2 = os.path.join('/Volumes/Blueman/HRSC_Jezero/', hrsc_cub)

tiff_image1 = os.path.splitext(cub_image1)[0] + '.tiff'
tiff_image2 = os.path.splitext(cub_image2)[0] + '.tiff'

i = gdal.Info(cub_image1, format='json')
print(i['coordinateSystem']['wkt'])
print('Equirectangular' in i['coordinateSystem']['wkt'])

resample = 'bilinear'
res = 50

PROJCS["Equirectangular Mars",
    GEOGCS["GCS_Mars",
        DATUM["D_Mars",
            SPHEROID["Mars_localRadius",3396190,0]],
        PRIMEM["Reference_Meridian",0],
        UNIT["degree",0.0174532925199433]],
    PROJECTION["Equirectangular"],
    PARAMETER["latitude_of_origin",0],
    PARAMETER["central_meridian",0],
    PARAMETER["standard_parallel_1",18.6],
    PARAMETER["false_easting",0],
    PARAMETER["false_northing",0]]
True


In [13]:
# Get all geo data for the two co-registered tiffs
cub_geo1 = GeoDataset(cub_image1)
cub_geo2 = GeoDataset(cub_image2)
overlap = cub_geo1.compute_overlap(cub_geo2)[0]

overlap_points1 = [cub_geo1.latlon_to_pixel(i[1], i[0]) for i in overlap][::2]
overlap_points2 = [cub_geo2.latlon_to_pixel(i[1], i[0]) for i in overlap][::2]

In [18]:
# if not os.path.exists(tiff_image_mod1):
ul, lr = overlap_points1

minx1, miny1= ul
maxx1, maxy1 = lr

fp = gdal.Translate(tiff_image1, cub_image1, srcWin=[minx1, miny1, maxx1 - minx1, maxy1 - miny1])
del(fp)

fp = gdal.Warp(tiff_image1, tiff_image1, 
               targetAlignedPixels=True, 
               xRes = res, yRes = res, 
               resampleAlg=resample)
del(fp)
# outputBounds = (minx1, miny1, maxx1, maxy1)

0 0
5760 9730


In [19]:
# if not os.path.exists(tiff_image_mod2):
ul, lr = overlap_points2

minx2, miny2= ul
maxx2, maxy2 = lr

fp = gdal.Translate(tiff_image2, cub_image2, srcWin=[minx2, miny2, maxx2 - minx2, maxy2 - miny2])
del(fp)

fp = gdal.Warp(tiff_image2, tiff_image2, 
               targetAlignedPixels=True, 
               xRes = res, yRes = res, 
               resampleAlg=resample)
del(fp)

In [ ]:
# Plot the images and there overlapping grid area
def show_initial_coregistration(source_image, destination_image):
    plt.figure(0, figsize=(10, 10))
    plt.imshow(source_image, alpha=.5, cmap='Greys')
    plt.imshow(destination_image, alpha=.5, cmap='Greys')
    plt.show()

# Show the quiver plot of the offsets
def display_quiver(comp_df, source_image, mask = [], scale = 100, scale_units = 'inches', **kwargs):
    if len(mask) != 0:
        comp_df = comp_df[mask]
    
    plt.figure(4, figsize=(20, 20))
    plt.imshow(source_image, cmap="Greys")
    plt.quiver(comp_df['destination_x'], comp_df['destination_y'], 
               -(comp_df['xoff']), (comp_df['yoff']),
               color = 'Red', scale = scale, scale_units = scale_units, **kwargs)
    plt.show()
    
# Given an index in the dataframe examine the before and after
# when the offset is applied
def examine_point(idx, size, comp_df, source_image, destination_image, mask = []):
    if len(mask) != 0:
        comp_df = comp_df[mask]
        
    plt.figure(2, figsize=(5, 5))
    plt.text(20, 50, 'Before Offset Correction', fontsize=12)
    x, y = int(comp_df.iloc[idx]['source_x']), int(comp_df.iloc[idx]['source_y'])
    plt.imshow(source_image[y - size:y + size, x - size:x + size], alpha = .5, cmap='Greys')

    x, y = int(comp_df.iloc[idx]['destination_x']), int(comp_df.iloc[idx]['destination_y'])
    plt.imshow(destination_image[y - size:y + size, x - size:x + size], alpha = .5, cmap='Greys')

    plt.figure(3, figsize=(5, 5))
    plt.text(20, 50, 'After Offset Correction', fontsize=12)
    x, y = int(comp_df.iloc[idx]['offset_source_x']), int(comp_df.iloc[idx]['offset_source_y'])
    plt.imshow(source_image[y - size: y + size, x - size: x + size], alpha = .5, cmap='Greys')

    x, y = int(comp_df.iloc[idx]['destination_x']), int(comp_df.iloc[idx]['destination_y'])
    plt.imshow(destination_image[y - size:y + size, x - size:x + size], alpha = .5, cmap='Greys')
    plt.show()
    
    offset_x, offset_y, corr = comp_df.iloc[idx][['xoff', 'yoff', 'corr']]
    print('X Offset: {}\nY Offset: {}\nCorrelation: {}'.format(offset_x, offset_y, corr))
    
def compute_homography(comp_df):
    x1 = np.array([*zip(comp_df['offset_source_x'].__array__(), comp_df['offset_source_y'].__array__())])
    x2 = np.array([*zip(comp_df['destination_x'].__array__(), comp_df['destination_y'].__array__())])
    H, mask = hg.compute_homography(x1, x2)
    
    return H, mask

# Apply the homography to the source image and display
def apply_homography(comp_df, source_image, destination_image):
    H, mask = compute_homography(comp_df)

    w, h = source_image.shape
    result = cv2.warpPerspective(source_image, H, (h, w))
    result[result == 0] = np.NAN

    plt.figure(0, figsize=(10, 10))
    plt.imshow(result, cmap='Greys', alpha = .5)
    plt.imshow(destination_image, cmap='Greys', alpha = .5)
    
def generate_point_grid(source_geo, destination_geo, source_raster, destination_raster, size):
    # Compute the overlap and get the corners now that
    # we have the geometry
    overlap_hull = source_geo.compute_overlap(destination_geo)[0]

    # Get the lats and lons of the assocaited corners
    overlap_lon = [i[0] for i in overlap_hull]
    overlap_lat = [i[1] for i in overlap_hull]

    # Define a ratio so the distrabution is even
    overlap_ratio = (max(overlap_lon) - min(overlap_lon)) / (max(overlap_lat) - min(overlap_lat))

    lon = np.linspace(min(overlap_lon) + .001, max(overlap_lon) - .001, size)
    lat = np.linspace(min(overlap_lat) + .001, max(overlap_lat) - .001, round(size/overlap_ratio))
    print('Generateing', len(lon), 'by', len(lat), 'point grid.')

    # Get the lat, lon position for the grid
    lonv, latv = np.meshgrid(lon, lat, sparse=True)

    coords = []

    # Begin looping over each point in the grid
    for lat_val in latv:
        for lon_val in lonv[0]:
            # Find the point in pixel space for each image and get the value
            x1, y1  = source_geo.latlon_to_pixel(lat_val[0], lon_val)
            x2, y2  = destination_geo.latlon_to_pixel(lat_val[0], lon_val)
            point_val1 = source_raster[y1 - 1, x1 - 1]
            point_val2 = destination_raster[y2 - 1, x2 - 1]

            # If either is zero then the point should be ignored
            # as it lies outside of the true overlap
            if point_val1 > 0 and point_val2 > 0:
                coords.append([x1, y1, x2, y2, lat_val[0], lon_val])

    # Build dataframe after grid contruction for data storage and 
    # ease of access
    df = pd.DataFrame(coords, columns = ['source_x', 
                                         'source_y', 
                                         'destination_x', 
                                         'destination_y', 
                                         'lat', 
                                         'lon'])
    return df

# The Meat and Potatoes of offset calculation
def compute_offsets(df, source_geo, destination_geo, template_size, search_size, corr_threshold=0.9):
    # Define a template size and a search space size
    s_img = source_geo

    d_img = destination_geo

    offsets = []
    bound_mask = []

    # Iterate through each point in the dataframe and calculate offsets
    print('Computing Offsets')
    for idx, row in df.iterrows():

        x, y = row['source_x'], row['source_y']
        s_template = sp.clip_roi(s_img, (x, y), template_size)


        x, y = row['destination_x'], row['destination_y']
        d_search = sp.clip_roi(d_img, (x, y), search_size)
        
        if not np.all(d_search) or not np.all(s_template):
            bound_mask.append(False)
        else:
            bound_mask.append(True)

        xoff, yoff, corr = sp.subpixel_offset(s_template, d_search)
        xoff, yoff, corr = xoff, yoff,corr
        # Apply the offsets to the source points and 
        # save those as well
        offset_source_x = row['source_x'] - xoff
        offset_source_y = row['source_y'] + yoff
        offsets.append([offset_source_x, offset_source_y, xoff, yoff, corr])
        sys.stdout.write('%s%s\r' % (round((idx/len(df) * 100)), '% complete'))
        sys.stdout.flush()
        
    off_df = pd.DataFrame(offsets, columns = ['offset_source_x', 'offset_source_y', 'xoff', 'yoff', 'corr'])
    comp_df = df.merge(off_df, left_index=True, right_index=True)[bound_mask]
    
    H, mask = compute_homography(comp_df.query('corr > {}'.format(corr_threshold)))
    return comp_df, H, mask

In [ ]:
tiff_geo1 = GeoDataset(tiff_image1)
tiff_geo2 = GeoDataset(tiff_image2)

# Setup and redefine all 0 values as NaNs
arr_image1 = tiff_geo1.read_array(1)
arr_image1[arr_image1 == 0] = np.NaN

arr_image2 = tiff_geo2.read_array(1)
arr_image2[arr_image2 == 0] = np.NaN

In [ ]:
plt.imshow(arr_image1)

In [ ]:
overlap_x = [i[1] for i in overlap]
overlap_y = [i[0] for i in overlap]

# x1 = [i[1] for i in tiff_geo1.latlon_corners]
# y1 = [i[0] for i in tiff_geo1.latlon_corners]

# x2 = [i[1] for i in tiff_geo2.latlon_corners]
# y2 = [i[0] for i in tiff_geo2.latlon_corners]

plt.figure(0, figsize=(10, 10))
# plt.imshow(arr_image1, extent = [min(y1), max(y1), min(x1), max(x1)], alpha = .5)
# plt.imshow(arr_image2, extent = [min(y2), max(y2), min(x2), max(x2)], alpha = .5)
plt.imshow(arr_image1, alpha = .5, cmap='plasma')
plt.imshow(arr_image2, alpha = .5, cmap='plasma')
# plt.scatter(overlap_y, overlap_x)

In [ ]:
show_initial_coregistration(arr_image1, arr_image2)

In [ ]:
# Generate a dataframe of points associated with a grid where each point
# in the grid is seperated by 
df = generate_point_grid(tiff_geo1, tiff_geo2, arr_image1, arr_image2, 20)

In [ ]:
comp_df, H, mask = compute_offsets(df, tiff_geo1, tiff_geo2, 25, 101)

In [ ]:
comp_df[['xoff', 'yoff']].plot(kind='box')

In [ ]:
comp_df[['xoff', 'yoff']].describe()

In [ ]:
H.round(12)

In [ ]:
# Add units
comp_df[['xoff', 'yoff', 'corr']].describe([.25, .5, .75, .99])

In [ ]:
# Add descriptions/units
plt.figure(0, figsize=(10, 10))
comp_df[['xoff', 'yoff']].plot(kind='box', figsize=(10, 10))

In [ ]:
plt.figure(1, figsize=(10, 10))
comp_df['corr'].plot(kind='box', figsize=(10, 10))

In [ ]:
ax1 = plt.axes

In [ ]:
display_quiver(comp_df, arr_image2, scale = 5)

In [ ]:
examine_point(13, 201, comp_df, arr_image1, arr_image2)

In [ ]:
apply_homography(comp_df[mask], arr_image1, arr_image2)

In [ ]:
comp_df